# Reshaping DataFrames
## Wide to long

In [2]:
using DataFrames # load package

In [3]:
x = DataFrame(id=[1,2,3,4], id2=[1,1,2,2], M1=[11,12,13,14], M2=[111,112,113,114])

4×4 DataFrame
 Row │ id     id2    M1     M2    
     │ Int64  Int64  Int64  Int64 
─────┼────────────────────────────
   1 │     1      1     11    111
   2 │     2      1     12    112
   3 │     3      2     13    113
   4 │     4      2     14    114

In [4]:
stack(x, [:M1, :M2], :id) # first pass measure variables and then id-variable

8×3 DataFrame
 Row │ id     variable  value 
     │ Int64  String    Int64 
─────┼────────────────────────
   1 │     1  M1           11
   2 │     2  M1           12
   3 │     3  M1           13
   4 │     4  M1           14
   5 │     1  M2          111
   6 │     2  M2          112
   7 │     3  M2          113
   8 │     4  M2          114

add view=true keyword argument to make a view; in that case columns of the resulting data frame share memory with columns of the source data frame, so the operation is potentially unsafe

In [5]:
# optionally you can rename columns
stack(x, ["M1", "M2"], "id", variable_name="key", value_name="observed")


8×3 DataFrame
 Row │ id     key     observed 
     │ Int64  String  Int64    
─────┼─────────────────────────
   1 │     1  M1            11
   2 │     2  M1            12
   3 │     3  M1            13
   4 │     4  M1            14
   5 │     1  M2           111
   6 │     2  M2           112
   7 │     3  M2           113
   8 │     4  M2           114

if second argument is omitted in stack , all other columns are assumed to be the id-variables

In [6]:
stack(x, Not([:id, :id2]))

8×4 DataFrame
 Row │ id     id2    variable  value 
     │ Int64  Int64  String    Int64 
─────┼───────────────────────────────
   1 │     1      1  M1           11
   2 │     2      1  M1           12
   3 │     3      2  M1           13
   4 │     4      2  M1           14
   5 │     1      1  M2          111
   6 │     2      1  M2          112
   7 │     3      2  M2          113
   8 │     4      2  M2          114

In [7]:
stack(x, Not([1, 2])) # you can use index instead of symbol

8×4 DataFrame
 Row │ id     id2    variable  value 
     │ Int64  Int64  String    Int64 
─────┼───────────────────────────────
   1 │     1      1  M1           11
   2 │     2      1  M1           12
   3 │     3      2  M1           13
   4 │     4      2  M1           14
   5 │     1      1  M2          111
   6 │     2      1  M2          112
   7 │     3      2  M2          113
   8 │     4      2  M2          114

In [8]:
x = DataFrame(id = [1,1,1], id2=['a','b','c'], a1 = rand(3), a2 = rand(3))

3×4 DataFrame
 Row │ id     id2   a1         a2       
     │ Int64  Char  Float64    Float64  
─────┼──────────────────────────────────
   1 │     1  a     0.50631    0.387321
   2 │     1  b     0.770559   0.188132
   3 │     1  c     0.0441943  0.817372

if stack is not passed any measure variables by default numeric variables are selected as measures

In [9]:
stack(x)

6×4 DataFrame
 Row │ id     id2   variable  value     
     │ Int64  Char  String    Float64   
─────┼──────────────────────────────────
   1 │     1  a     a1        0.50631
   2 │     1  b     a1        0.770559
   3 │     1  c     a1        0.0441943
   4 │     1  a     a2        0.387321
   5 │     1  b     a2        0.188132
   6 │     1  c     a2        0.817372

here all columns are treated as measures:

In [10]:
stack(DataFrame(rand(3,2), :auto))

6×2 DataFrame
 Row │ variable  value     
     │ String    Float64   
─────┼─────────────────────
   1 │ x1        0.612335
   2 │ x1        0.0162968
   3 │ x1        0.0850522
   4 │ x2        0.77054
   5 │ x2        0.758132
   6 │ x2        0.874147

In [11]:
df = DataFrame(rand(3,2), :auto)
df.key = [1,1,1]
mdf = stack(df) # duplicates in key are silently accepted

6×3 DataFrame
 Row │ key    variable  value    
     │ Int64  String    Float64  
─────┼───────────────────────────
   1 │     1  x1        0.400451
   2 │     1  x1        0.499192
   3 │     1  x1        0.600074
   4 │     1  x2        0.393869
   5 │     1  x2        0.716906
   6 │     1  x2        0.631193

## Long to wide

In [12]:
x = DataFrame(id = [1,1,1], id2=['a','b','c'], a1 = rand(3), a2 = rand(3))
y = stack(x)

6×4 DataFrame
 Row │ id     id2   variable  value     
     │ Int64  Char  String    Float64   
─────┼──────────────────────────────────
   1 │     1  a     a1        0.433666
   2 │     1  b     a1        0.707949
   3 │     1  c     a1        0.123797
   4 │     1  a     a2        0.0750425
   5 │     1  b     a2        0.271233
   6 │     1  c     a2        0.466925

In [13]:
unstack(y, :id2, :variable, :value) # stndard unstack with a specified key

3×3 DataFrame
 Row │ id2   a1        a2        
     │ Char  Float64?  Float64?  
─────┼───────────────────────────
   1 │ a     0.433666  0.0750425
   2 │ b     0.707949  0.271233
   3 │ c     0.123797  0.466925

In [14]:
unstack(y, :variable, :value) # all other columns are treated as keys

3×4 DataFrame
 Row │ id     id2   a1        a2        
     │ Int64  Char  Float64?  Float64?  
─────┼──────────────────────────────────
   1 │     1  a     0.433666  0.0750425
   2 │     1  b     0.707949  0.271233
   3 │     1  c     0.123797  0.466925

In [15]:
# all columns other than named :variable and :value are treated as keys
unstack(y)

3×4 DataFrame
 Row │ id     id2   a1        a2        
     │ Int64  Char  Float64?  Float64?  
─────┼──────────────────────────────────
   1 │     1  a     0.433666  0.0750425
   2 │     1  b     0.707949  0.271233
   3 │     1  c     0.123797  0.466925

In [16]:
# you can rename the unstacked columns
unstack(y, renamecols=n->string("unstacked_", n))

3×4 DataFrame
 Row │ id     id2   unstacked_a1  unstacked_a2 
     │ Int64  Char  Float64?      Float64?     
─────┼─────────────────────────────────────────
   1 │     1  a         0.433666     0.0750425
   2 │     1  b         0.707949     0.271233
   3 │     1  c         0.123797     0.466925

In [17]:
df = stack(DataFrame(rand(3,2), :auto))


6×2 DataFrame
 Row │ variable  value     
     │ String    Float64   
─────┼─────────────────────
   1 │ x1        0.0667908
   2 │ x1        0.0811233
   3 │ x1        0.3445
   4 │ x2        0.826426
   5 │ x2        0.26148
   6 │ x2        0.225545

In [18]:
unstack(df, :variable, :value) # unable to unstack when no key column is present


LoadError: ArgumentError: No key column found

In [19]:
df = DataFrame(key=[1, 1, 2], variable=["a", "b", "a"], value=1:3)


3×3 DataFrame
 Row │ key    variable  value 
     │ Int64  String    Int64 
─────┼────────────────────────
   1 │     1  a             1
   2 │     1  b             2
   3 │     2  a             3

In [21]:
unstack(df, :variable, :value)

2×3 DataFrame
 Row │ key    a       b       
     │ Int64  Int64?  Int64?  
─────┼────────────────────────
   1 │     1       1        2
   2 │     2       3  missing 

In [22]:
unstack(df, :variable, :value, fill=0)

2×3 DataFrame
 Row │ key    a      b     
     │ Int64  Int64  Int64 
─────┼─────────────────────
   1 │     1      1      2
   2 │     2      3      0